In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import importlib
import mr_rrn, data_aux
from tensorflow.contrib import learn
importlib.reload(data_aux)

<module 'data_aux' from '/Users/bjornmattsson/Documents/MrRRN/data_aux.py'>

In [3]:
# Settings
#examples=50000
num_steps = 20
num_seq = 6
n_hidden_enc = 300
n_hidden_con = 300
n_hidden_dec = 300
batch_size = 128
learning_rate = 0.001
#embedding_shape = [3, 3] # [voc_size, emb_dim]

In [ ]:
# Load data
#X = data_aux.generate_test_data_sequence(examples=examples, num_seq=num_seq, num_steps=num_steps)
#x_data, y_data = X[:,:-1,:], X[:,1:,:]
x_data, y_data, vocab_processor = data_aux.generate_data_ubuntu(max_utterances = num_seq, max_tokens = num_steps)
batches = data_aux.batch_iter([x_data, y_data],batch_size=batch_size, num_epochs=5)

# Build graph
timer = datetime.now()
tf.reset_default_graph()
(vocab_input, x_input, y_input, total_loss, train_step) = mr_rrn.build_graph([len(vocab_processor.vocabulary_), 300], num_seq, num_steps, batch_size, n_hidden_enc, n_hidden_con, n_hidden_dec, learning_rate, gradient_clipping = 0.)
timer = datetime.now() - timer
print('Time build graph: ', timer)

# Train
W_embedding = data_aux.pretrained_embedding(vocab_processor)
#W_embedding = np.eye(3)
acc_loss = 0
with tf.Session() as sess:
    timer = datetime.now()
    sess.run(tf.initialize_all_variables(), feed_dict={vocab_input : W_embedding})
    timer = datetime.now() - timer
    print('Time init graph: ', timer)
    timer = datetime.now()
    for i, batch in enumerate(batches):
        x_batch, y_batch = batch
        feed_dict = {x_input : x_batch,
                     y_input : y_batch
                     }
        loss, _ = sess.run([total_loss, train_step], feed_dict=feed_dict)
        print(loss)
timer = datetime.now() - timer
print('Time run graph: ', timer)

Time build graph:  0:00:23.906104
Time init graph:  0:00:14.256172
9.62334
4.55464
1.94668
2.28608
2.53452
2.46169
2.21429
1.80503
1.70123
1.78298
1.76741
1.56656
1.75154
1.53372
1.40993
1.5445
